### 1. Tổng quan đồ án

Đại dịch COVID-19 do virus SARS-CoV-2 gây ra vẫn đang tiếp tục diễn ra mạnh mẽ, lây nhiễm cho hàng trăm triệu người và giết chết hàng triệu người. Theo Trung tâm Kiểm soát và Phòng ngừa Dịch bệnh (CDC), nhiễm COVID-19 lây truyền chủ yếu qua các giọt đường hô hấp tạo ra khi con người hít thở, nói chuyện, ho hoặc hắt hơi. Đeo khẩu trang là phương pháp chính, hiệu quả và tiện lợi để ngăn chặn 80% các bệnh nhiễm trùng đường hô hấp. Khẩu trang làm giảm sự lây truyền bằng cách ngăn không cho nước bọt phát tán quá xa vào bầu khí quyển. Do đó, nhu cầu ngày càng tăng đối với các hệ thống tự động có thể phát hiện xem mọi người đeo khẩu trang hoặc không đeo khẩu trang.  

Đồ án này được thiết kế để giải quyết vấn đề nêu trên.

Phạm vi bài toán: 
- Mục tiêu phát hiện đối tượng mặt người đeo khẩu trang , không đeo khẩu trang, trong hình ảnh, video, để cung cấp khả năng giám sát hiệu quả cho các bệnh viện, sân bay, địa điểm thể thao và địa điểm bán lẻ. 

Nguồn gốc đồ án: 
   - Sử dụng deep learning models (YOLOv5) , code từ cuộc thi DATA COMPETITION do fsoft tổ chức
https://github.com/fsoft-ailab/Data-Competition 
   

Tập dữ liệu liên quan: 
   - https://www.kaggle.com/datasets/anntfx10327/facemaskpj

Sau khi tổng hợp dữ liệu phát hiện đối tượng có:
   - 9204 file hình ảnh, ngoài đường, trong nhà, có hình ảnh chụp đối diện, mặt bên có hình ảnh chụp từ trên xuống. Tập dữ liệu đã được gán 2 nhãn No mask va Mask với các số 0,1 tương ứng với mỗi nhãn.
   
   - 9204 file .txt có nhãn và bouding box (x center, y center, boxes width, boxes height) của các đối tượng thuộc khoảng 0-1    

|   | No mask: 0 | Mask: 1 | Total file| 
|---|---|---|---|  
| Train  | 5142  | 3697  | 4602 |
| Val  | 2595  | 2107  | 2301 |
| Test  | 3101  | 1891  | 2301 |


Dữ liệu bỏ vào trong thư mục **data** theo cấu trúc sau:
```
data
├── images
│   ├── train
│   │   ├── train_img1.jpg
│   │   ├── train_img2.jpg
│   │   └── ...
│   │   
│   └── val
│   |   ├── val_img1.jpg
│   |   ├── val_img2.jpg
│   |   └── ...
|   │   
│   └── test
│       ├── test_img1.jpg
│       ├── test_img2.jpg
│       └── ...
│   
└── labels
    ├── train
    │   ├── train_img1.txt
    │   ├── train_img2.txt
    │   └── ...
    │   
    └── val
    |   ├── val_img1.txt
    |   ├── val_img2.txt
    |   └── ...
    │   
    └── test
        ├── test_img1.txt
        ├── test_img2.txt
        └── ...
```
[LINK DATASET](https://drive.google.com/drive/folders/13Awh0Vy3ikBgeZZVLvIWy6u1VcCppkP0?usp=sharing)

### 2. Xác định phạm vi đồ án

Bài toán: 
- phát hiện đối tượng (object detection), phát hiện khuôn mặt người ở trong ảnh có đeo khẩu trang không đeo khẩu trang, đeo khẩu trang không đúng cách.

Chiến lược giải bài toán: 
- Sử dụng deep learning model yolov5,  pretrain yolov5s


### 3. Metrics

Metrics được dùng để đo lường hiệu suất: mAP@0.5

- Chi tiết về  mAP@0.5 metrics: 
  - Giá trị mAP (mean Average Precision) khi IoU = 0.5 ,tính trung bình AP cho tất cả các lớp.
  - AP phần diện tích phía dưới đường biểu diễn precision-recall 

- Tại sao mAP là một metric tốt để đánh giá mô hình Object Detection???
    - Mối quan hệ giữa precision – recall giúp mAP đánh giá được về độ chính xác của nhiệm vụ phân loại.
    - Do precision – recall thay đổi khi ngưỡng IoU thay đổi (ngưỡng để predict một bbox là class nào). Do đó, tại một giá trị IoU xác định, có thể đo / so sánh độ tốt của các mô hình (ví dụ: mAP@0.5 = 70 –> tại IoU = 0.5, AP của mô hình là 70%).

### 4. Tiền xử lý dữ liệu

- Các bước tiền xử lý dữ liệu:
    - resize ảnh về  kích thước 640 x 640
    - chia dữ liệu thành 3 tập train/val/test
    - xử lý dữ liệu về định dạng file .txt yolo 
    ```class x center, y center, boxes width, boxes height``` từ các nguồn dữ liệu .xml, .json
    


### 5. Triển khai thuật toán 

- Sử dụng YOLOv5s và cũng sử dụng một mô hình được đào tạo trước được đào tạo với bộ dữ liệu mặt nạ dễ dàng để giảm đáng kể thời gian đào tạo.
- Chỉnh sửa đường dẫn train/val/test trong file `data_cfg.yaml` để đọc đúng đường dẫn file data
- Sử dụng: `batch-size=16` 
- đào tạo:  `epochs=10` theo dõi `mAP@0.5` của tập kiểm định `val`.

In [21]:
%%writefile ./data/data.yaml

train: /content/drive/MyDrive/production/mlp5/Data/data/images/train/
val: /content/drive/MyDrive/production/mlp5/Data/data/images/val/
test: /content/drive/MyDrive/production/mlp5/Data/data/images/test/

# number of classes
nc: 2

# class names
names: ['no_mask', 'mask']

Writing ./data/data.yaml


In [ ]:
!python train.py --batch-size 16 --epochs 10 --img 640 --data ./data/data.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --cache 

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=./data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-97-g2da6866 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, t

In [ ]:
!python val.py --weights runs/train/exp/weights/best.pt --data ./data/data.yaml --img 640 --iou 0.5 --task train --batch-size 16

val: data=./data/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=train, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-97-g2da6866 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
train: Scanning '/content/drive/MyDrive/mlp5/yolov5/data/labels/train.cache' images and labels... 4602 found, 0 missing, 0 empty, 0 corrupt: 100% 4602/4602 [00:00<?, ?it/s]
train: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/train/000140.jpg: corrupt JPEG restored and saved
train: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/train/000567.jpg: corrupt JPEG restored and saved
train: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/train/000855.jpg: corrupt

In [ ]:
!python val.py --weights runs/train/exp/weights/best.pt --data ./data/data.yaml --img 640 --iou 0.5 --task val --batch-size 16

val: data=./data/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-97-g2da6866 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '/content/drive/MyDrive/mlp5/yolov5/data/labels/val.cache' images and labels... 2301 found, 0 missing, 0 empty, 0 corrupt: 100% 2301/2301 [00:00<?, ?it/s]
val: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/val/000250.jpg: corrupt JPEG restored and saved
val: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/val/000867.jpg: corrupt JPEG restored and saved
val: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/val/001074.jpg: corrupt JPEG restored and

In [ ]:
!python val.py --batch-size 16 --weights runs/train/exp/weights/best.pt --data ./data/data.yaml --img 640 --iou 0.5 --task test

val: data=./data/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-97-g2da6866 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/content/drive/MyDrive/mlp5/yolov5/data/labels/test' images and labels...2301 found, 0 missing, 0 empty, 0 corrupt: 100% 2301/2301 [10:59<00:00,  3.49it/s]
test: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/test/000256.jpg: corrupt JPEG restored and saved
test: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/test/000307.jpg: corrupt JPEG restored and saved
test: WARNING: /content/drive/MyDrive/mlp5/yolov5/data/images/test/000627.jpg: corrupt JPEG re

### 6. Cải thiện thuật toán

- do model overfitting nên ta sử dụng kỹ thuật tăng cường dữ liệu::
    - chuyển đổi hình ảnh sang dạng HSV:    
        ```hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
        hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
        hsv_v: 0.4  # image HSV-Value augmentation (fraction)```
    - translate: 0.1  # image translation (+/- fraction)
    - scale: 0.9  # image scale (+/- gain)
    - fliplr: 0.5  # image flip left-right (probability)
    - mosaic: 1.0  # image mosaic (probability)
    - mixup: 0.1  # image mixup (probability)
    - copy_paste: 0.1  # segment copy-paste (probability)

In [24]:
!python train.py --batch-size 16 --epochs 10 --img 640 --data ./data/data.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --cache --hyp data/hyps/hyp.scratch-high.yaml

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=./data/data.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-99-g2181ef3 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, t

In [25]:
!python val.py --weights runs/train/exp/weights/best.pt --data ./data/data.yaml --img 640 --iou 0.5 --task train --batch-size 16

val: data=./data/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=train, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-99-g2181ef3 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
train: Scanning '/content/drive/MyDrive/production/mlp5/Data/data/labels/train.cache' images and labels... 4602 found, 0 missing, 0 empty, 0 corrupt: 100% 4602/4602 [00:00<?, ?it/s]
train: WARNING: /content/drive/MyDrive/production/mlp5/Data/data/images/train/000140.jpg: corrupt JPEG restored and saved
train: WARNING: /content/drive/MyDrive/production/mlp5/Data/data/images/train/000567.jpg: corrupt JPEG restored and saved
train: WARNING: /content/drive/MyDrive/production/mlp5/Data/d

In [26]:
!python val.py --weights runs/train/exp/weights/best.pt --data ./data/data.yaml --img 640 --iou 0.5 --task val --batch-size 16

val: data=./data/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-99-g2181ef3 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '/content/drive/MyDrive/production/mlp5/Data/data/labels/val.cache' images and labels... 2301 found, 0 missing, 0 empty, 0 corrupt: 100% 2301/2301 [00:00<?, ?it/s]
val: WARNING: /content/drive/MyDrive/production/mlp5/Data/data/images/val/000250.jpg: corrupt JPEG restored and saved
val: WARNING: /content/drive/MyDrive/production/mlp5/Data/data/images/val/000867.jpg: corrupt JPEG restored and saved
val: WARNING: /content/drive/MyDrive/production/mlp5/Data/data/images/val/0

In [27]:
!python val.py --weights runs/train/exp/weights/best.pt --data ./data/data.yaml --img 640 --iou 0.5 --task test --batch-size 16

val: data=./data/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-99-g2181ef3 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/content/drive/MyDrive/production/mlp5/Data/data/labels/test' images and labels...2301 found, 0 missing, 0 empty, 0 corrupt: 100% 2301/2301 [11:37<00:00,  3.30it/s]
test: WARNING: /content/drive/MyDrive/production/mlp5/Data/data/images/test/000256.jpg: corrupt JPEG restored and saved
test: WARNING: /content/drive/MyDrive/production/mlp5/Data/data/images/test/000307.jpg: corrupt JPEG restored and saved
test: WARNING: /content/drive/MyDrive/production/mlp5/Data/data/ima

### 7. Đánh giá và kiểm định chất lượng mô hình 

Đánh giá kết quả. Sau 6 epochs:

||mAP@.5 |sau cải thiện|Images|
|---|---|---|---|
|Train|0.916|0.914|4602|
|Val|0.867|0.881|2301|
|Test|0.85|0.871|2301| 

8. Trien khai


In [17]:
import numpy as np
import cv2
import torch
import time

PATH = 'weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=PATH)
d = {0:"no_mask", 1:"mask"}
def draw_bbox(img, bbox, fontScale=0.8):
    for dt in bbox:
        name = d[int(dt[-1])] + f": {float(dt[-2]):.2f}" 
        pt1 = int(dt[0]), int(dt[1])
        pt2 = int(dt[2]), int(dt[3])
        if int(dt[-1]) == 1:
            color=(255,0,0)
        else:
            color=(0,0,255)
            
        cv2.rectangle(img, pt1=pt1, pt2=pt2, color=color, thickness=2)
        cv2.putText(img,text=name,org=pt1,fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=fontScale,color=color,thickness=2)
    return img

Using cache found in /home/annt/.cache/torch/hub/ultralytics_yolov5_master
fatal: not a git repository (or any of the parent directories): .git
YOLOv5 🚀 2022-4-6 torch 1.10.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 


In [19]:
use_mp4 = False
capture = cv2.VideoCapture(0)
if use_mp4:
    path_id = 'tm.mp4'
    path_out = 'output.mp4'
    capture = cv2.VideoCapture(path_id)
    frame_width = int(capture.get(3))
    frame_height = int(capture.get(4))
    frame_size = (frame_width,frame_height)
    
    out = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc(*'mp4v'), 20, frame_size)

if (capture.isOpened() == False):
    print("Error opening the video file")

else:
    fps = capture.get(5)
    print('Frames per second : ', fps,'FPS')

    frame_count = capture.get(7)
    print('Frame count : ', frame_count)

start = time.time_ns()
frame_count = 0
total_frames = 0
fps = -1
while capture.isOpened():

    _, frame = capture.read()
    if frame is None:
        print("End of stream")
        break

    frame_count += 1
    total_frames += 1

    model.conf = 0.4
    model.iou = 0.5 
    results = model([frame[..., ::-1]], size=320) 
    bbox = results.xyxy[0]
    frame = draw_bbox(frame, bbox)
    
    if frame_count >= 30:
        end = time.time_ns()
        fps = 1000000000 * frame_count / (end - start)
        frame_count = 0
        start = time.time_ns()

    if fps > 0:
        fps_label = "FPS: %.2f" % fps
        cv2.putText(frame, fps_label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
    cv2.imshow("output", frame)
    if use_mp4:
        out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("DONE!")
        break

if use_mp4:    
    out.release()
capture.release()

cv2.destroyAllWindows()
end = time.time()
print("Total frames: " + str(total_frames))

Frames per second :  30.0 FPS
Frame count :  -1.0


QObject::moveToThread: Current thread (0x55a106965ab0) is not the object's thread (0x55a1017a7cb0).
Cannot move to target thread (0x55a106965ab0)

QObject::moveToThread: Current thread (0x55a106965ab0) is not the object's thread (0x55a1017a7cb0).
Cannot move to target thread (0x55a106965ab0)

QObject::moveToThread: Current thread (0x55a106965ab0) is not the object's thread (0x55a1017a7cb0).
Cannot move to target thread (0x55a106965ab0)

QObject::moveToThread: Current thread (0x55a106965ab0) is not the object's thread (0x55a1017a7cb0).
Cannot move to target thread (0x55a106965ab0)

QObject::moveToThread: Current thread (0x55a106965ab0) is not the object's thread (0x55a1017a7cb0).
Cannot move to target thread (0x55a106965ab0)

QObject::moveToThread: Current thread (0x55a106965ab0) is not the object's thread (0x55a1017a7cb0).
Cannot move to target thread (0x55a106965ab0)

QObject::moveToThread: Current thread (0x55a106965ab0) is not the object's thread (0x55a1017a7cb0).
Cannot move to tar

DONE!
Total frames: 70
